## Prerequisites

In [ ]:
 !pip install transformers

In [ ]:
!pip install trafilatura

In [ ]:
!pip install wikipedia

## Imports

In [ ]:
from transformers import pipeline
qa_pipeline = pipeline(
            "question-answering",
            model="henryk/bert-base-multilingual-cased-finetuned-polish-squad2",
            tokenizer="henryk/bert-base-multilingual-cased-finetuned-polish-squad2",
        )

In [ ]:
from googlesearch import search
from bs4 import BeautifulSoup

import requests
import trafilatura
import wikipedia

## Source

In [ ]:
def get_urls(query):
    urls = list(search(query, tld='com', lang="pl", start=0, stop=3, pause=1.0))
    # print(urls)
    urls = [url for url in urls if "#" not in url]
    seen_urls = set()
    res_urls = []
    for url in urls:
      if url[:20] not in seen_urls:
        seen_urls.add(url[:20])
        res_urls.append(url)
    return res_urls

def get_context(url):
    wikipedia.set_lang("pl")
    if "wikipedia" in url:
        r = requests.get(url)
        html = r.text
        soup = BeautifulSoup(html, "html.parser")
        title = soup.find("title").text[:-32]
        wiki_page = wikipedia.page(title)
        context = wiki_page.content
    else:
        downloaded = trafilatura.fetch_url(url)
        context = trafilatura.extract(downloaded)
    return context

def find_potential_answers(question):
    urls = get_urls(question)
    answers = []
    seen_answers = set()
    for url in urls:
        context = get_context(url)
        if context:
            context = context[:500]
            ans = qa_pipeline({"context": context, "question": question})
            if ans['answer'] not in seen_answers:
              seen_answers.add(ans['answer'])
            else:
              return answers
            answers.append(ans)
    return answers

def determine_best_answer(answers):
  return (max(answers, key=lambda x:x['score']))['answer']

def find_answer(question):
  potential_answers = find_potential_answers(question)
  return determine_best_answer(potential_answers)

In [ ]:
find_answer("Kim jest Adam Małysz?")

In [ ]:
# find_answer("Co robi Robert Makłowicz?")

In [ ]:
# find_answer("Co jest stolicą Polski?")

duckduck go try


In [ ]:
import pprint

In [ ]:
import requests

# query = "Co to jest żyrafa?"
query = "What is the UFO?"

r = requests.get("https://api.duckduckgo.com",
    params = {
        "q": query,
        "format": "json"
    })

data = r.json()

print(data)

print("Abstract")
pprint.pprint(data["Abstract"])